In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-aug-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-aug-2022/train.csv
/kaggle/input/tabular-playground-series-aug-2022/test.csv


In [2]:
df_train = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/train.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/test.csv')

df_train['df_train'] = 1
df_test['df_train'] = 0
df_test['failure'] = np.nan
df = pd.concat([df_train, df_test], axis = 0)

df = pd.get_dummies(df)

from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
df_value = imp_mean.fit_transform(df)
df = pd.DataFrame(data = df_value, columns = df.columns)

df_train = df.loc[df['df_train'] == 1, :].drop(columns = ['df_train'], axis=1)
df_test = df.loc[df['df_train'] == 0, :].drop(columns = ['df_train', 'failure'], axis=1)


In [3]:
from sklearn.model_selection import train_test_split

X = df_train.loc[:, df_train.columns != 'failure']
y = df_train['failure']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.8007150922092585


In [4]:
result = clf.predict_proba(df_test)[:, 1]
my_submission = pd.DataFrame({'id': df_test.id.astype('int32'), 'failure': result})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

In [5]:
my_submission

,id,failure
26570,26570,0.30
26571,26571,0.25
26572,26572,0.23
26573,26573,0.28
26574,26574,0.36
...,...,...
47340,47340,0.27
47341,47341,0.21
47342,47342,0.20
47343,47343,0.32
